# Notebook to automatize the writing of different ferroelectric structures for beyond-binary information encoding

## Author: Marti Checa

Developed winter 2024-2025

In [1]:
from sys import exit
import time
from scipy import signal
import numpy as np
import matplotlib.pyplot as plt
from nifpga import Session
import time
import pylab as pl
from IPython import display
import win32com.client  
import os
from joblib import Parallel, delayed
import pythoncom
import sidpy as sid
from skimage.registration import phase_cross_correlation
from scipy.ndimage import fourier_shift
import warnings
warnings.filterwarnings('ignore')

import h5py
import pyNSID

from scipy.special import erf

# import acquition.py
from Acquisition_v0_2 import Acquisition   # include the Acquistion_v0.py in the same directory

### Start executable - activeX needs it running before it can connect

In [2]:
#exe_path = r"C:/Users/Asylum User/Desktop/User data/Mani/07-05-22/V3 PyAE 062822 01 (1)/V3 PyAE 062822 01.exe"
exe_path = r'C:\STAFF Software\Marti_AE\BEPyAE 051123 01\\BEPyAE.exe'
os.startfile(exe_path)
time.sleep(1)

### Connect to executable via active X

In [3]:
labview = win32com.client.Dispatch("BEPyAE.Application")
#slash is critical here: double slash, tilt to left
VI = labview.getvireference(r'C:\STAFF Software\Marti_AE\BEPyAE 051123 01\\BEPyAE.exe\\FPGA PyScanner\\FPGA_PyScanner_01.vi') 

At this point, we set up the center of the image to be our zero zero with the "Go there" instruction of AR software

## Define Important functions

In [4]:
def move_tip_to_pos(fin_x,fin_y,trans_t):
    line_scan_cluster = list(VI.getcontrolvalue('line_scan_control_cluster'))#get the line scan cluster
    line_scan_cluster[2] = fin_x 
    line_scan_cluster[3] = fin_y 
    line_scan_cluster[4] = trans_t 
    VI.setcontrolvalue('line_scan_control_cluster',tuple(line_scan_cluster))#send the new parameters
    VI.setcontrolvalue('make_cur_pos_start_pos', True);#tell it to use current pos as starting pos
    VI.setcontrolvalue('do_probe_move_update', True);#update params of move tip
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_probe_move_update')        
    VI.setcontrolvalue('do_probe_move', True) #Move the tip to new location
    moving = True
    while (moving==True):
        moving = VI.getcontrolvalue('do_probe_move')
    
    print('Tip moved to ('+str(fin_x) + ',' + str(fin_y) + ')' )
       
def change_tip_bias(tip_bias):        
    VI.setcontrolvalue('set_AO2_V',tip_bias); # changed value
    #VI.setcontrolvalue('set_AO0_V',VI.getcontrolvalue('get_AO0_V')); # changed value
    #VI.setcontrolvalue('set_AO1_V',VI.getcontrolvalue('get_AO1_V')); # changed value
    VI.setcontrolvalue('do_set_output_val', 1);
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_set_output_val')
        
    print('Tip bias changed to '+str(tip_bias)+'V')
        
def do_write_spiral_scan(center_x,center_y,hh_writting_bias,
                   hh_sp_number,hh_Vx,hh_Vy,hh_cycles,
                   hh_spiral_time,hh_direction,hh_return):
    
    print("Performing spiral scans...")
    #Defininig spiral scan type
    spiral_scan_cluster = list(VI.getcontrolvalue('spiral_scan_control_cluster'))#get the spiral raster scan cluster
    spiral_scan_cluster[1] = hh_Vx #change X size
    spiral_scan_cluster[3] = hh_Vy #change X size
    spiral_scan_cluster[4] = hh_cycles #change cycles
    spiral_scan_cluster[5] = hh_spiral_time #change duration
    spiral_scan_cluster[7] = hh_direction #change direction
    spiral_scan_cluster[8] = hh_return #change return type
    VI.setcontrolvalue('spiral_scan_control_cluster',tuple(spiral_scan_cluster))#send the new parameters
    
    #set up spiral center
    VI.setcontrolvalue('scan_x_offset_V', center_x)
    VI.setcontrolvalue('scan_y_offset_V', center_y)
    VI.setcontrolvalue('set_AO0_V',center_x); # changed value
    VI.setcontrolvalue('set_AO1_V',center_y); # changed value
    VI.setcontrolvalue('scan_type', 1)#change to spiral scan
    
    #update params
    VI.setcontrolvalue('do_scan_update', 1) #need to tell PyScanner to update parameters internally 
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_scan_update')
        
    TipBias_vec=np.full((int(hh_spiral_time*1E6)),hh_writting_bias)#generating vector of tip biases
    VI.setcontrolvalue('AO2_V',TipBias_vec.tolist())
    time.sleep(0.5)
        
    for k in range(hh_sp_number):
        #Performing scan  
        VI.setcontrolvalue('do_scan', 1)# do scan
        #keep checking if scan is complete
        scanning = True
        while (scanning==True):
            scanning = VI.getcontrolvalue('do_scan')
    
    print("Spiral finished")
    
def Set_Setpoint(Setpoint):

    VI.setcontrolvalue('connected_to_cypher', True);#connect to cypher
    VI.setcontrolvalue('set_contact_setpoint_V', Setpoint);#set setpoint
    VI.setcontrolvalue('set_contact_setpoint', True);#connect to cypher
    
    changing_setpoint = False
    while (changing_setpoint==False):
        changing_setpoint = VI.getcontrolvalue('set_contact_setpoint')
    
    print('Setpoint changed to '+str(Setpoint)+' V')
    
def Square_spiral(center_x,center_y,hh_writting_bias,alter_bias,hh_sp_number,hh_Vx,hh_Vy,hh_cycles,
                   hh_spiral_time,hh_direction,hh_return):
    
    spiral_outer_radius_x = hh_Vx
    spiral_inner_radius_x = 0
    spiral_outer_radius_y = hh_Vy
    spiral_inner_radius_y = 0
    IO_rate = 1E6
    N_cycles = hh_cycles
    duration = hh_spiral_time
    dose_distribution = 1
    direction_opt = hh_direction  # 0 = clockwise spiral out, 1 = counter spiral out, 2 = clockwise spiral in, 3 = counter spiral in
    return_opt = hh_return  # 0 = pigtail return to center, 1 = spiral back to center
    cut_off_frequency = 2000  # [Hz] smooth corners of non-circular spirals

    N_points = int(duration * IO_rate)
    t = np.linspace(0, 1, N_points)
    w = np.linspace(-IO_rate / 2, IO_rate / 2 - IO_rate / N_points, N_points)

    switch_return_opt = return_opt
    switch_direction_opt = direction_opt

    if switch_return_opt == 0:  # pigtail return to center
        amp_vec_x = (spiral_outer_radius_x - spiral_inner_radius_x) * t ** dose_distribution + spiral_inner_radius_x
        amp_vec_y = (spiral_outer_radius_y - spiral_inner_radius_y) * t ** dose_distribution + spiral_inner_radius_y
        x1 = amp_vec_x * np.arcsin(np.sin(2 * np.pi * N_cycles * t))
        y1 = amp_vec_y * np.arcsin(np.cos(2 * np.pi * N_cycles * t))
        a = 1 / N_cycles / 32  # 1/32 of the spiral rotation is spent returning smoothly (erf-like) to the center
        transition_x = (1 - (erf((t - 1 + a * 2) / a) + 1) / 2) * (1 - spiral_inner_radius_x / spiral_outer_radius_x) + spiral_inner_radius_x / spiral_outer_radius_x
        transition_y = (1 - (erf((t - 1 + a * 2) / a) + 1) / 2) * (1 - spiral_inner_radius_y / spiral_outer_radius_y) + spiral_inner_radius_y / spiral_outer_radius_y
        x_vec = x1 * transition_x
        y_vec = y1 * transition_y
    elif switch_return_opt == 1:  # spiral back to center
        tf = t[0::2]
        tr = t[-2::-2]
        amp_vecf_x = (spiral_outer_radius_x - spiral_inner_radius_x) * tf ** dose_distribution + spiral_inner_radius_x
        amp_vecr_x = (spiral_outer_radius_x - spiral_inner_radius_x) * tr ** dose_distribution + spiral_inner_radius_x
        amp_vecf_y = (spiral_outer_radius_y - spiral_inner_radius_y) * tf ** dose_distribution + spiral_inner_radius_y
        amp_vecr_y = (spiral_outer_radius_y - spiral_inner_radius_y) * tr ** dose_distribution + spiral_inner_radius_y
        xf = amp_vecf_x * np.arcsin(np.sin(2 * np.pi * N_cycles * tf))
        xr = -amp_vecr_x * np.arcsin(np.sin(2 * np.pi * N_cycles * tr))
        yf = amp_vecf_y * np.arcsin(np.cos(2 * np.pi * N_cycles * tf))
        yr = amp_vecr_y * np.arcsin(np.cos(2 * np.pi * N_cycles * tr))
        x_vec = np.concatenate((xf, xr))
        y_vec = np.concatenate((yf, yr))

    filter = np.exp(-(w / cut_off_frequency) ** 2)
    filter = np.fft.fftshift(filter)
    x_vec = np.real(np.fft.ifft(np.fft.fft(x_vec) * filter))
    y_vec = np.real(np.fft.ifft(np.fft.fft(y_vec) * filter))

    start_ind = np.argmin(np.abs(x_vec + 1j * y_vec))
    x_vec = np.roll(x_vec, -start_ind + 1) + center_x
    y_vec = np.roll(y_vec, -start_ind + 1) + center_y

    if switch_direction_opt == 1:
        x_vec = -x_vec
    elif switch_direction_opt == 2:
        x_vec = -x_vec
        x_vec = np.flip(x_vec)
        y_vec = np.flip(y_vec)
    elif switch_direction_opt == 3:
        x_vec = np.flip(x_vec)
        y_vec = np.flip(y_vec)

    sector_bias_mat = np.array([[0, 90, hh_writting_bias],
                            [90, 180, -hh_writting_bias],
                            [180, 270, hh_writting_bias],
                            [270, 359.9, -hh_writting_bias]])

    bias_vec = np.zeros_like(x_vec)
    a_vec = np.arctan2(y_vec, x_vec) * 180 / np.pi + 180  # angle of scan path

    for k1 in range(sector_bias_mat.shape[0]):
        ind = np.where((a_vec > sector_bias_mat[k1, 0]) & (a_vec < sector_bias_mat[k1, 1]))
        bias_vec[ind] = sector_bias_mat[k1, 2]
        
    if alter_bias==0:
        bias_vec[:]=hh_writting_bias

    #%% plot trajectory
    plt.figure(1)
    plt.clf()
    plt.plot(x_vec, color='blue')
    plt.plot(y_vec, color='red')
    plt.plot(bias_vec, color='green')
    plt.title('x,y,bias vs. time')

    # plot position over time
    cmap = plt.get_cmap('rainbow')
    N_color_segments = 1000
    colors = cmap(np.linspace(0, 1, N_color_segments))
    plt.figure(2)
    plt.clf()
    line_ind = np.linspace(0, len(x_vec)-1, N_color_segments, dtype = int)
    for i in range(N_color_segments-1):
        plt.plot(x_vec[line_ind[i]:line_ind[i+1]],y_vec[line_ind[i]:line_ind[i+1]],color=colors[i])
    plt.axis('square')
    plt.title('position over time')
    plt.show()

    # plot bias vs position
    if alter_bias!=0:
        colors = cmap(np.linspace(0, 1, N_color_segments))
        plt.figure(2)
        plt.clf()
        line_ind = np.linspace(0, len(x_vec)-1, N_color_segments, dtype = int)
        norm_bias1 = (bias_vec-bias_vec.min())/(bias_vec-bias_vec.min()).max()*(N_color_segments-1)
        norm_bias = norm_bias1.astype(int)
        for i in range(N_color_segments-1):
            plt.plot(x_vec[line_ind[i]:line_ind[i+1]],y_vec[line_ind[i]:line_ind[i+1]],color=colors[norm_bias[line_ind[i]]])
        plt.axis('square')
        plt.title('angle dependent bias')
        plt.show()
    
    #Load the vectors
    VI.setcontrolvalue('UI_AO0_V',x_vec.tolist())
    VI.setcontrolvalue('UI_AO1_V',y_vec.tolist())
    VI.setcontrolvalue('UI_AO2_V',V_vec.tolist())

    #update the scan parameters and wait until finished
    VI.setcontrolvalue('do_scan_update', 1)
    updating = True
    while (updating==True):
        updating = VI.getcontrolvalue('do_scan_update')

    #We move tip to the first point to scan
    move_tip_to_pos(center_x,+center_y,1)
    
    # we engage
    Set_Setpoint(Setpoint=1)

    #Perform the scan 
    print('Writting square spiral...')
    for k in range(hh_sp_number):
        #do scan and wait until finished   
        VI.setcontrolvalue('do_scan', 1)
        scanning = True
        while (scanning==True):
            scanning = VI.getcontrolvalue('do_scan')

    # we withdraw
    Set_Setpoint(Setpoint=-10)
    #We move tip back to 0,0
    move_tip_to_pos(0,0,1)
    

### Here we define our alphabet (in this case I will program only until base 6)

simbol_0 #this will be a cw spiral scan with negative bias

simbol_1 #this will be a ccw spiral scan with negative bias

simbol_2 #this will be an outter cw spiral scan with negative bias and an inner cw spiral scan with positive bias 

simbol_3 #this will be an outter ccw spiral scan with negative bias and an inner cw spiral scan with positive bias 

simbol_4 #this will be an outter cw spiral scan with negative bias and an inner ccw spiral scan with positive bias 

simbol_5 #this will be an outter ccw spiral scan with negative bias and an inner ccw spiral scan with positive bias 


In [100]:
#Now we define the functions:

def simbol_0(center_x,center_y,Wbias,hh_sp_number,OUTradius,INradius,hh_cycles,hh_spiral_time,hh_return=0):
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=-Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=OUTradius,hh_Vy=factorY*OUTradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=0,
                         hh_return=hh_return)
    
def simbol_1(center_x,center_y,Wbias,hh_sp_number,OUTradius,INradius,hh_cycles,hh_spiral_time,hh_return=0):
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=-Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=OUTradius,hh_Vy=factorY*OUTradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=1,
                         hh_return=hh_return)
    
def simbol_2(center_x,center_y,Wbias,hh_sp_number,OUTradius,INradius,hh_cycles,hh_spiral_time,hh_return=0):
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=-Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=OUTradius,hh_Vy=factorY*OUTradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=0,
                         hh_return=hh_return)
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=INradius,hh_Vy=factorY*INradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=0,
                         hh_return=hh_return)
    
def simbol_3(center_x,center_y,Wbias,hh_sp_number,OUTradius,INradius,hh_cycles,hh_spiral_time,hh_return=0):
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=-Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=OUTradius,hh_Vy=factorY*OUTradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=1,
                         hh_return=hh_return)
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=INradius,hh_Vy=factorY*INradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=0,
                         hh_return=hh_return)

def simbol_4(center_x,center_y,Wbias,hh_sp_number,OUTradius,INradius,hh_cycles,hh_spiral_time,hh_return=0):
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=-Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=OUTradius,hh_Vy=factorY*OUTradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=0,
                         hh_return=hh_return)
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=INradius,hh_Vy=factorY*INradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=1,
                         hh_return=hh_return)
    
def simbol_5(center_x,center_y,Wbias,hh_sp_number,OUTradius,INradius,hh_cycles,hh_spiral_time,hh_return=0):
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=-Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=OUTradius,hh_Vy=factorY*OUTradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=1,
                         hh_return=hh_return)
    
    do_write_spiral_scan(center_x=center_x, center_y=center_y,
                         hh_writting_bias=Wbias,
                         hh_sp_number=hh_sp_number,
                         hh_Vx=INradius,hh_Vy=factorY*INradius,
                         hh_cycles=hh_cycles,
                         hh_spiral_time=hh_spiral_time,
                         hh_direction=1,
                         hh_return=hh_return)

## This cell generates a grid of ceertain dimansions with 1 specified simbol.

In [182]:
# Calculate the center of the grid
rows=1
cols=4
spacing = 2

center_x_global = cols * spacing / 2
center_y_global = rows * spacing / 2

move_tip_to_pos(0,0,1)

Wbias=9 #magnitude
OUTradius=0.6 #magnitude
INradius=0.2 #magnitude
cycles=16
time_per_spiral=1
number_of_spirals=30
factorY=1

for i in range(rows):
    for j in range(cols):
    
        center_x = j * spacing - center_x_global + spacing / 2
        center_y = center_y_global - i * spacing - spacing / 2
        
        move_tip_to_pos(center_x,center_y,1)
        
        simbol_0(center_x,center_y,Wbias,number_of_spirals,OUTradius,INradius,cycles,time_per_spiral)

move_tip_to_pos(0,0,1)

Tip moved to (0,0)
Tip moved to (-3.0,0.0)
Performing spiral scans...
Spiral finished
Tip moved to (-1.0,0.0)
Performing spiral scans...
Spiral finished
Tip moved to (1.0,0.0)
Performing spiral scans...
Spiral finished
Tip moved to (3.0,0.0)
Performing spiral scans...
Spiral finished
Tip moved to (0,0)


## Here we encode information in an array of topologies (treating them like an alphabet)

The script works as follows:

1. Experimental parameteres are chos

In [120]:
#Now we define the parameters:

Wbias=8 #magnitude
OUTradius=0.4 #magnitude
INradius=0.2 #magnitude
cycles=16
time_per_spiral=1
number_of_spirals=30
factorY=1
spacing = 0.97

In [146]:
# Here we write the

def base_conversion(num, base):
    """
    Converts a number to the given base and returns it as a string.
    """
    if num == 0:
        return "0"
    digits = []
    while num > 0:
        digits.append(int(num % base))
        num //= base
    return ''.join(str(d) for d in digits[::-1])


# Ask the user for the base and the word
base = int(input("Enter the base to use (2-6): "))
word = input("Enter the word to write: ")
    
# Create a numpy array to store the representations
ascii_values = [ord(char) for char in word]
base_representations = [base_conversion(value, base) for value in ascii_values]
    
# Determine the maximum length for the array
max_length = max(len(rep) for rep in base_representations)
    
# Fill the numpy array with the base representations
base_matrix = np.full((len(word), max_length), fill_value=' ', dtype='<U1')
for i, rep in enumerate(base_representations):
    base_matrix[i, -len(rep):] = list(rep)  # Right-align the representation
    
# Return the results
print("\nBase representations:")
print(base_matrix)
print("\nDimensions of the matrix:")
print(base_matrix.shape)


Enter the base to use (2-6): 6
Enter the word to write: Rama Vasudevan

Base representations:
[['2' '1' '4']
 ['2' '4' '1']
 ['3' '0' '1']
 ['2' '4' '1']
 [' ' '5' '2']
 ['2' '2' '2']
 ['2' '4' '1']
 ['3' '1' '1']
 ['3' '1' '3']
 ['2' '4' '4']
 ['2' '4' '5']
 ['3' '1' '4']
 ['2' '4' '1']
 ['3' '0' '2']]

Dimensions of the matrix:
(14, 3)


In [147]:
#base_matrix=[[0,1,2],[3,4,5]]

print("\nBase representations:")
print(base_matrix)
print("\nDimensions of the matrix:")
print(base_matrix.shape)


Base representations:
[['2' '1' '4']
 ['2' '4' '1']
 ['3' '0' '1']
 ['2' '4' '1']
 [' ' '5' '2']
 ['2' '2' '2']
 ['2' '4' '1']
 ['3' '1' '1']
 ['3' '1' '3']
 ['2' '4' '4']
 ['2' '4' '5']
 ['3' '1' '4']
 ['2' '4' '1']
 ['3' '0' '2']]

Dimensions of the matrix:
(14, 3)


In [148]:
# Now we program the execution of the writting

def write_word(base_matrix, spacing, OUTradius):
    """
    Writes the word using the machine's functions based on the base matrix.

    Args:
        base_matrix (list of lists): Matrix of characters in base representation.
        spacing (float): Distance between adjacent characters in the grid.
        OUTradius (float): Radius of the characters.

    Returns:
        list of lists: Matrix of (center_x, center_y) pairs.
    """
    rows = len(base_matrix)
    cols = len(base_matrix[0])
    
    # Calculate the center of the grid
    center_x_global = cols * spacing / 2
    center_y_global = rows * spacing / 2
    
    # Initialize a matrix for the coordinates
    coordinates = []
    
    move_tip_to_pos(0 ,0 ,1)
    
    # Loop through the base matrix
    for i in range(rows):
        row_coords = []
        for j in range(cols):
            # Calculate the center position for each character
            center_x = j * spacing - center_x_global + spacing / 2
            center_y = center_y_global - i * spacing - spacing / 2
            row_coords.append((center_x, center_y))
            
            move_tip_to_pos(center_x ,center_y ,1)
            
            # Get the character to write
            char = base_matrix[i][j]
            if char != ' ':  # Ignore empty spaces
                simbol_index = int(char)  # Convert character to integer
                globals()[f"simbol_{simbol_index}"](center_x, center_y, 
                                                    Wbias,
                                                    number_of_spirals,
                                                    OUTradius,INradius,
                                                    cycles,
                                                    time_per_spiral)
                
                time.sleep(0.5)   
      
        coordinates.append(row_coords)
    
    move_tip_to_pos(0 ,0 ,1)
    
    return coordinates

In [149]:
# Execute the writing process
coordinates_matrix = write_word(base_matrix, spacing, OUTradius)
    
print("\nCoordinates matrix (center_x, center_y):")
for row in coordinates_matrix:
    print(row)

Tip moved to (0,0)
Tip moved to (-0.9700000000000001,6.305)
Performing spiral scans...
Spiral finished
Performing spiral scans...
Spiral finished
Tip moved to (-1.1102230246251565e-16,6.305)
Performing spiral scans...


KeyboardInterrupt: 

In [151]:
move_tip_to_pos(0,0,1)

Tip moved to (0,0)
